<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 03 - Creating and Querying a Data Mart

In this tutorial you will learn how to create and query a Data Mart using Virtualized Data in a SQL Server big data cluster. 

Wide World Importers is interested in ingesting the data from web logs from an HDFS source where they have been streamed. They want to be able to analyze the traffic to see if there is a pattern in time, products or locations. 

The web logs, however, are refreshed periodically. WWI would like to keep the logs in local storage to do deeper analysis. 

In this Jupyter Notebook you'll create a location to store the log files as a SQL Server Table in the SQL Data Pool, and then fill it by creating an External Table that reads HDFS.

In [1]:
IF NOT EXISTS
(
  SELECT 1
  FROM sys.external_data_sources
  WHERE
    name = 'SqlDataPool'
)
BEGIN
  CREATE EXTERNAL DATA SOURCE SqlDataPool
  WITH (LOCATION = 'sqldatapool://controller-svc/default');
END

In [4]:
IF NOT EXISTS
(
    SELECT 1
    FROM sys.external_tables
    WHERE
        name = N'web_clickstream_clicks_data_pool'
)
BEGIN
    CREATE EXTERNAL TABLE dbo.web_clickstream_clicks_data_pool
    (
        wcs_click_date_sk BIGINT,
        wcs_click_time_sk BIGINT,
        wcs_sales_sk BIGINT,
        wcs_item_sk BIGINT,
        wcs_web_page_sk BIGINT,
        wcs_user_sk BIGINT
    )
    WITH
    (
        DATA_SOURCE = SqlDataPool,
        DISTRIBUTION = ROUND_ROBIN
    );
END
GO

In [5]:
IF NOT EXISTS
(
    SELECT 1
    FROM sys.external_file_formats
    WHERE
        name = N'csv_file'
)
BEGIN
    CREATE EXTERNAL FILE FORMAT csv_file
    WITH
    (
        FORMAT_TYPE = DELIMITEDTEXT,
        FORMAT_OPTIONS
        (
            FIELD_TERMINATOR = ',',
            STRING_DELIMITER = '"',
            FIRST_ROW = 2, 
            USE_TYPE_DEFAULT = True
        )
    )
END
GO

/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS
(
    SELECT 1
    FROM sys.external_tables
    WHERE
        name = 'web_clickstreams_hdfs'
)
BEGIN
    CREATE EXTERNAL TABLE dbo.web_clickstreams_hdfs
    (
        wcs_click_date_sk BIGINT,
        wcs_click_time_sk BIGINT,
        wcs_sales_sk BIGINT,
        wcs_item_sk BIGINT,
        wcs_web_page_sk BIGINT,
        wcs_user_sk BIGINT
    )
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/web_logs',
        FILE_FORMAT = csv_file
    );
END

In [6]:
INSERT INTO dbo.web_clickstream_clicks_data_pool
SELECT
   hdfs.wcs_click_date_sk,
   hdfs.wcs_click_time_sk,
   hdfs.wcs_sales_sk,
   hdfs.wcs_item_sk,
   hdfs.wcs_web_page_sk,
   hdfs.wcs_user_sk
FROM web_clickstreams_hdfs hdfs;

In [7]:
SELECT
    COUNT(*) AS TotalRecords
FROM [dbo].[web_clickstream_clicks_data_pool];

SELECT TOP(10) *
FROM [dbo].[web_clickstream_clicks_data_pool];

## Next Steps: Continue on to Working with Spark and ETL

Now you're ready to open the next Python Notebook - `notebook_04.ipynb` - to learn how to create and work with Spark and Extracting, Transforming and Loading data.